### SQL Databases

In [1]:
## create sample SQLite Database
import os
import sqlite3

In [2]:
os.makedirs("data/database", exist_ok=True)

In [4]:
## create sample database
connection = sqlite3.connect("data/database/company.db")
cursor = connection.cursor()

In [5]:
# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS employees
                 (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)''')

In [6]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
                 (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [7]:
# Insert sample data
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]


In [ ]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [9]:
cursor.execute("Select * from employees")

In [10]:
connection.commit()
connection.close()

## Database Content Extraction

In [11]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

/home/aniruddha/Projects/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
## MEthod1 SQLDATABASE Utility
db = SQLDatabase.from_uri("sqlite:///data/database/company.db")

## get Database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable DDL:")
print(db.get_table_info())

Tables: ['employees', 'projects']

Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


In [14]:
from typing import List
from langchain_core.documents import Document
# Method 2: Custom SQL to Document conversion
print("\n2️⃣ Custom SQL Processing")

def sql_to_documents(db_path:str)-> List[Document]:
    """Convert SQL Database To documents with context"""
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()

    documents = []

     # Example: Join employees and projects
    cursor.execute("""
        SELECT e.name, e.role, p.name as project_name, p.status
        FROM employees e
        JOIN projects p ON e.id = p.lead_id
    """)
    rows = cursor.fetchall()
    rel_content = "Employee-Project Relationships:\n\n"

    for row in rows:
        rel_content += f"Employee: {row[0]}, Role: {row[1]}, Project: {row[2]}, Status: {row[3]}\n"
    
    doc = Document(
        page_content=rel_content,
        metadata={
            "source": db_path, 
            "type": "employee_project_relationships"
        }
    )

    documents.append(doc)
    connection.close()
    return documents



2️⃣ Custom SQL Processing


In [16]:
sql_docs = sql_to_documents("data/database/company.db")
sql_docs


[Document(metadata={'source': 'data/database/company.db', 'type': 'employee_project_relationships'}, page_content='Employee-Project Relationships:\n\nEmployee: John Doe, Role: Senior Developer, Project: RAG Implementation, Status: Active\nEmployee: Jane Smith, Role: Data Scientist, Project: Data Pipeline, Status: Completed\nEmployee: Mike Johnson, Role: Product Manager, Project: Customer Portal, Status: Planning\nEmployee: Jane Smith, Role: Data Scientist, Project: ML Platform, Status: Active\n')]